# 3. Modelagem

In [1]:
# Configurações de visualização
%matplotlib inline
%config inlineBackend.figure_formats = ['svg']

# Importação das Bibliotecas

# Manipulação
import pandas as pd
import numpy as np

# Machine Learning

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb

#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_validate

from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.compose import ColumnTransformer

import joblib

# Configurações de visualização do dataset
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 90)

In [2]:
# Dados de Treino
df_train4 = pd.read_csv('cars_train4.csv', encoding='utf-8')

In [3]:
# Tamanho dos dados de treino
df_train4.shape

(29396, 31)

In [4]:
# Visualização dos dados de treino
df_train4.head()

,num_fotos,marca,modelo,modelos_por_marca,modelos_por_tipo,modelos_por_anofabricacao,modelos_por_estado,modelos_por_anunciante,ano_de_fabricacao,hodometro,cambio,tipo,blindado,cor,num_portas,cilindrada,tipo_combustivel,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,dono_aceita_troca,veiculo_único_dono,ipva_pago,veiculo_licenciado,revisoes_concessionaria,revisoes_dentro_agenda,garantia_de_fábrica,preco
0,8,NISSAN,KICKS,11,345,191,402,443,2017,67772.0,CVT,Sedã,N,Branco,4,1.6,Outros,PF,Rio de Janeiro,São Paulo (SP),Pessoa Física,False,False,Não Aceita Troca,Mais de um dono,IPVA pago,Licenciado,Todas as revisões feitas pela concessionária,Revisões feitas fora da agenda do carro,Sem Garantia de fábrica,74732.590084
1,8,JEEP,COMPASS,6,345,191,214,443,2017,62979.0,Automática,Sedã,N,Branco,4,2.0,Flex,PF,Belo Horizonte,Minas Gerais (MG),Pessoa Física,False,False,Aceita troca,Mais de um dono,IPVA pago,Não Licenciado,Revisões feitas fora da concessionária,Revisões feitas fora da agenda do carro,Sem Garantia de fábrica,81965.332634
2,16,KIA,SORENTO,11,345,202,402,296,2018,44070.0,Automática,Sedã,N,Preto,4,2.4,Gasolina,PJ,Santos,São Paulo (SP),Loja,True,False,Aceita troca,Mais de um dono,IPVA Não Pago,Não Licenciado,Revisões feitas fora da concessionária,Revisões feitas fora da agenda do carro,Sem Garantia de fábrica,162824.814472
3,14,VOLKSWAGEN,AMAROK,27,164,233,402,296,2013,85357.0,Automática,Picape,N,Branco,4,2.0,Diesel,PJ,Sorocaba,São Paulo (SP),Loja,True,True,Aceita troca,Mais de um dono,IPVA pago,Licenciado,Revisões feitas fora da concessionária,Revisões feitas fora da agenda do carro,Sem Garantia de fábrica,123681.358857
4,8,SSANGYONG,KORANDO,1,175,233,245,443,2013,71491.0,Automática,Utilitário esportivo,N,Preto,4,2.0,Diesel,PF,Rio de Janeiro,Rio de Janeiro (RJ),Pessoa Física,False,False,Não Aceita Troca,Mais de um dono,IPVA Não Pago,Não Licenciado,Todas as revisões feitas pela concessionária,Todas as revisões feitas pela agenda do carro,Garantia de fábrica,82419.763891


**Divisão em Dados de Treino e Validação**

In [5]:
# Separação das variáveis preditoras da variável alvo
X = df_train4.drop(columns='preco') 
Y = df_train4['preco']

In [6]:
# Divisão do Dataset em treino e validação
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.3, random_state=18)

In [7]:
# Tamanho de cada um dos conjuntos do Dataset após a divisão
X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape

((20577, 30), (8819, 30), (20577,), (8819,))

## 3.1 - Pipeline 1 - Algoritmos Variados

Ao realizar testes com algoritmos baseados em distância, o Count Encoding é uma opção adequada. Esse método substitui cada valor categórico pela contagem de vezes que ele ocorre no conjunto de dados, fornecendo uma medida da frequência absoluta de cada categoria.

Algoritmos baseados em distância levam em consideração as diferenças nos valores das variáveis ao calcular distâncias entre amostras. Com o Count Encoding, categorias menos frequentes terão contagens menores, enquanto categorias mais frequentes terão contagens maiores. Essas contagens podem refletir as distâncias relativas entre as categorias, auxiliando no cálculo das distâncias entre as amostras.


In [8]:
# Pipeline 1

preprocessor1 = ColumnTransformer(
transformers=[
    ('Padronização', StandardScaler(), ['num_fotos', 'ano_de_fabricacao', 'hodometro', 'num_portas',
                                        'cilindrada', 'modelos_por_marca', 'modelos_por_tipo',
                                          'modelos_por_anofabricacao', 'modelos_por_estado', 'modelos_por_anunciante']),
    ('Count Encoder', ce.CountEncoder(), ['marca', 'modelo', 'cambio', 'tipo', 'cor',
                                                'cidade_vendedor', 'estado_vendedor', 'anunciante', 'tipo_combustivel',
                                                'blindado', 'tipo_vendedor', 'entrega_delivery', 'troca', 'dono_aceita_troca',
                                                  'veiculo_único_dono', 'revisoes_concessionaria', 'ipva_pago', 'veiculo_licenciado',
                                                    'garantia_de_fábrica', 'revisoes_dentro_agenda'])])

In [9]:
# Visualizar as transformações do Pipeline 1

# Aplicar pré-processamento usando o pipeline
X_train_p1 = preprocessor1.fit_transform(X_train)

# Converter o resultado em DataFrame
X_train_p1 = pd.DataFrame(X_train_p1, columns=preprocessor1.get_feature_names_out())

# Exibir as primeiras 5 linhas do DataFrame
X_train_p1.head()

,Padronização__num_fotos,Padronização__ano_de_fabricacao,Padronização__hodometro,Padronização__num_portas,Padronização__cilindrada,Padronização__modelos_por_marca,Padronização__modelos_por_tipo,Padronização__modelos_por_anofabricacao,Padronização__modelos_por_estado,Padronização__modelos_por_anunciante,Count Encoder__marca,Count Encoder__modelo,Count Encoder__cambio,Count Encoder__tipo,Count Encoder__cor,Count Encoder__cidade_vendedor,Count Encoder__estado_vendedor,Count Encoder__anunciante,Count Encoder__tipo_combustivel,Count Encoder__blindado,Count Encoder__tipo_vendedor,Count Encoder__entrega_delivery,Count Encoder__troca,Count Encoder__dono_aceita_troca,Count Encoder__veiculo_único_dono,Count Encoder__revisoes_concessionaria,Count Encoder__ipva_pago,Count Encoder__veiculo_licenciado,Count Encoder__garantia_de_fábrica,Count Encoder__revisoes_dentro_agenda
0,-0.661791,0.802955,-0.348634,0.17699,-0.309301,-0.568848,0.890564,-0.349198,0.831341,0.744475,1101,936,15672,11435,14588,589,11380,12583,10464,20410,12536,False,False,15222,7269,6408,13623,10985,3005,4166
1,2.219099,0.802955,-0.134131,0.17699,-0.830866,0.487678,-1.094476,-0.349198,-2.749005,-0.931055,2102,593,15672,3415,14588,35,76,6791,10464,20410,8041,False,False,15222,7269,14169,13623,10985,17572,16411
2,-0.661791,0.064292,0.69221,0.17699,0.038409,-0.370749,-1.094476,0.39542,-0.692004,0.744475,695,47,15672,3415,14588,4054,1781,12583,10464,20410,12536,False,False,5355,13308,14169,13623,9592,17572,16411
3,-0.661791,0.802955,-1.264244,0.17699,-0.309301,-0.568848,0.890564,-0.349198,0.831341,0.744475,1101,936,15672,11435,1123,4293,11380,12583,10464,20410,12536,False,False,15222,7269,6408,13623,9592,3005,16411
4,1.066743,-4.860125,3.295334,-5.668798,-0.483156,0.355612,-1.162534,-3.228386,0.831341,-0.931055,3207,19,3446,3355,14588,3,11380,6791,10464,20410,8041,False,False,15222,13308,14169,6954,9592,17572,16411


In [10]:
# Lista de modelos
modelos = []
modelos.append(('KNeighborsRegressor', KNeighborsRegressor()))
modelos.append(('SVR', SVR()))
modelos.append(('LinearRegression', LinearRegression()))


# Definir as métricas de avaliação
scoring = {
    'r2': 'r2',
    'mae': make_scorer(mean_absolute_error, greater_is_better=False),
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False),
    'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False)
}

# Avaliando cada modelo em um loop 
nomes = []
resultados = []
resultados_mean = []
resultados_std = []
mae_mean = []
mae_std = []
mse_mean = []
mse_std = []
rmse_mean = []
rmse_std = []
mape_mean = []
mape_std = []

# Definindo os valores para o número de folds 
num_folds = 10
seed = 18

for nome, modelo in modelos:
    clf = Pipeline(steps=[('preprocessor', preprocessor1), ('modelos', modelo)])
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_validate(clf, X_train, Y_train, cv=kfold, scoring=scoring)
    resultados.append(cv_results)
    resultados_mean.append(cv_results['test_r2'].mean() * 100)
    resultados_std.append(cv_results['test_r2'].std() * 100)
    mae_mean.append(-cv_results['test_mae'].mean())
    mae_std.append(cv_results['test_mae'].std())
    mse_mean.append(-cv_results['test_mse'].mean())
    mse_std.append(cv_results['test_mse'].std())
    rmse_mean.append(-cv_results['test_rmse'].mean())
    rmse_std.append(cv_results['test_rmse'].std())
    mape_mean.append(-cv_results['test_mape'].mean())
    mape_std.append(cv_results['test_mape'].std())
    nomes.append(nome)

# Criar o dataframe final
model_result = pd.DataFrame({
    'Modelo': nomes,
    'R-Squared Mean': resultados_mean,
    'R-Squared STD': resultados_std,
    'MAE Mean': mae_mean,
    'MAE STD': mae_std,
    'MSE Mean': mse_mean,
    'MSE STD': mse_std,
    'RMSE Mean': rmse_mean,
    'RMSE STD': rmse_std,
    'MAPE Mean': mape_mean,
    'MAPE STD': mape_std
})

# Arredondar valores para duas casas decimais
model_result = model_result.round(2)

model_result = model_result.sort_values('R-Squared Mean', ascending=False)
model_result = model_result.reset_index(drop=True)

model_result

,Modelo,R-Squared Mean,R-Squared STD,MAE Mean,MAE STD,MSE Mean,MSE STD,RMSE Mean,RMSE STD,MAPE Mean,MAPE STD
0,LinearRegression,48.12,4.48,40509.29,944.63,3.417265e+09,4.997571e+08,58308.93,4163.45,0.39,0.01
1,KNeighborsRegressor,39.03,3.26,41925.30,1078.18,4.000143e+09,3.587800e+08,63183.83,2818.83,0.37,0.01
2,SVR,-5.22,0.96,57215.64,1392.46,6.907141e+09,5.433074e+08,83044.95,3267.58,0.51,0.01


## 3.2 - Pipeline 2 - Algoritmos Baseados em Árvores

Para algoritmos baseados em árvores, o Target Encoding é uma abordagem recomendada. Ele codifica as categorias com base nas estatísticas do target, como a média. Isso permite que as árvores de decisão capturem relações entre as categorias e o valor de destino. O Target Encoding é especialmente útil para construir regras nas árvores de decisão, pois fornece informações sobre a relação entre as categorias e o target. Além disso, as árvores de decisão são robustas a valores discrepantes e não requerem normalização dos dados, o que torna o Target Encoding uma escolha adequada para variáveis categóricas em algoritmos baseados em árvores.

In [11]:
# Pipeline 1

preprocessor2 = ColumnTransformer(
transformers=[
    ('Target Encoder', ce.TargetEncoder(), ['marca', 'modelo', 'cambio', 'tipo', 'cor',
                                                'cidade_vendedor', 'estado_vendedor', 'anunciante', 'tipo_combustivel',
                                                'blindado', 'tipo_vendedor', 'entrega_delivery', 'troca', 'dono_aceita_troca',
                                                  'veiculo_único_dono', 'revisoes_concessionaria', 'ipva_pago', 'veiculo_licenciado',
                                                    'garantia_de_fábrica', 'revisoes_dentro_agenda'])], remainder='passthrough')

In [12]:
# Visualizar as transformações do Pipeline 2

# Aplicar pré-processamento usando o pipeline
preprocessor2.fit(X_train, Y_train)

# Transformar os dados de treinamento
X_train_p2 = preprocessor2.transform(X_train)

# Converter o resultado em DataFrame
X_train_p2 = pd.DataFrame(X_train_p2, columns=preprocessor2.get_feature_names_out())

# Exibir as primeiras 5 linhas do DataFrame
X_train_p2.head()

,Target Encoder__marca,Target Encoder__modelo,Target Encoder__cambio,Target Encoder__tipo,Target Encoder__cor,Target Encoder__cidade_vendedor,Target Encoder__estado_vendedor,Target Encoder__anunciante,Target Encoder__tipo_combustivel,Target Encoder__blindado,Target Encoder__tipo_vendedor,Target Encoder__entrega_delivery,Target Encoder__troca,Target Encoder__dono_aceita_troca,Target Encoder__veiculo_único_dono,Target Encoder__revisoes_concessionaria,Target Encoder__ipva_pago,Target Encoder__veiculo_licenciado,Target Encoder__garantia_de_fábrica,Target Encoder__revisoes_dentro_agenda,remainder__num_fotos,remainder__modelos_por_marca,remainder__modelos_por_tipo,remainder__modelos_por_anofabricacao,remainder__modelos_por_estado,remainder__modelos_por_anunciante,remainder__ano_de_fabricacao,remainder__hodometro,remainder__num_portas,remainder__cilindrada
0,121933.386343,124208.728844,145934.192978,118054.461719,130103.12777,140812.523918,129412.457964,138698.11017,100166.365356,132299.591446,138719.415931,False,False,136903.357138,140237.800222,154243.403661,132245.917245,131669.875735,168622.097922,151984.73005,8.0,13.0,345.0,161.0,402.0,443.0,2020.0,47168.0,4.0,1.6
1,93447.871613,84674.474833,145934.192978,93276.253584,130103.12777,113214.469448,95896.042126,121862.026809,100166.365356,132299.591446,123790.447931,False,False,136903.357138,140237.800222,123226.329258,132245.917245,131669.875735,126774.196771,128037.121719,18.0,29.0,170.0,161.0,33.0,296.0,2020.0,54158.0,4.0,1.0
2,127995.017098,64708.719753,145934.192978,93276.253584,130103.12777,132062.116401,134182.184089,138698.11017,100166.365356,132299.591446,138719.415931,False,False,121464.554168,128869.628803,123226.329258,132245.917245,134277.731961,126774.196771,128037.121719,8.0,16.0,170.0,191.0,245.0,443.0,2017.0,81086.0,4.0,2.0
3,121933.386343,124208.728844,145934.192978,118054.461719,196740.06668,126065.280337,129412.457964,138698.11017,100166.365356,132299.591446,138719.415931,False,False,136903.357138,140237.800222,154243.403661,132245.917245,134277.731961,168622.097922,128037.121719,8.0,13.0,345.0,161.0,402.0,443.0,2020.0,17331.0,4.0,1.6
4,118793.638951,108645.487538,80578.353447,157107.451443,130103.12777,126268.026107,129412.457964,121862.026809,100166.365356,132299.591446,123790.447931,False,False,136903.357138,128869.628803,123226.329258,134138.547497,134277.731961,126774.196771,128037.121719,14.0,27.0,164.0,45.0,402.0,296.0,1997.0,165914.0,2.0,1.4


In [13]:
# Lista de modelos
modelos = []
modelos.append(('DecisionTreeRegressor', DecisionTreeRegressor(random_state=18)))
modelos.append(('RandomForestRegressor', RandomForestRegressor(random_state=18)))
modelos.append(('XGBRegressor', XGBRegressor(random_state=18)))
modelos.append(('LGBMRegressor', lgb.LGBMRegressor(random_state=18)))
modelos.append(('AdaBoostRegressor', AdaBoostRegressor(random_state=18)))
modelos.append(('GradientBoostingRegressor', GradientBoostingRegressor(random_state=18)))


# Definir as métricas de avaliação
scoring = {
    'r2': 'r2',
    'mae': make_scorer(mean_absolute_error, greater_is_better=False),
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False),
    'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False)
}

# Avaliando cada modelo em um loop 
nomes = []
resultados = []
resultados_mean = []
resultados_std = []
mae_mean = []
mae_std = []
mse_mean = []
mse_std = []
rmse_mean = []
rmse_std = []
mape_mean = []
mape_std = []

# Definindo os valores para o número de folds 
num_folds = 10
seed = 18

for nome, modelo in modelos:
    clf = Pipeline(steps=[('preprocessor', preprocessor2), ('modelos', modelo)])
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_validate(clf, X_train, Y_train, cv=kfold, scoring=scoring)
    resultados.append(cv_results)
    resultados_mean.append(cv_results['test_r2'].mean() * 100)
    resultados_std.append(cv_results['test_r2'].std() * 100)
    mae_mean.append(-cv_results['test_mae'].mean())
    mae_std.append(cv_results['test_mae'].std())
    mse_mean.append(-cv_results['test_mse'].mean())
    mse_std.append(cv_results['test_mse'].std())
    rmse_mean.append(-cv_results['test_rmse'].mean())
    rmse_std.append(cv_results['test_rmse'].std())
    mape_mean.append(-cv_results['test_mape'].mean())
    mape_std.append(cv_results['test_mape'].std())
    nomes.append(nome)

# Criar o dataframe final
model_result = pd.DataFrame({
    'Modelo': nomes,
    'R-Squared Mean': resultados_mean,
    'R-Squared STD': resultados_std,
    'MAE Mean': mae_mean,
    'MAE STD': mae_std,
    'MSE Mean': mse_mean,
    'MSE STD': mse_std,
    'RMSE Mean': rmse_mean,
    'RMSE STD': rmse_std,
    'MAPE Mean': mape_mean,
    'MAPE STD': mape_std
})

# Arredondar valores para duas casas decimais
model_result = model_result.round(2)

model_result = model_result.sort_values('R-Squared Mean', ascending=False)
model_result = model_result.reset_index(drop=True)

model_result


,Modelo,R-Squared Mean,R-Squared STD,MAE Mean,MAE STD,MSE Mean,MSE STD,RMSE Mean,RMSE STD,MAPE Mean,MAPE STD
0,LGBMRegressor,73.72,1.65,27128.48,717.30,1.729188e+09,2.105743e+08,41505.72,2542.15,0.23,0.01
1,RandomForestRegressor,72.86,1.72,27353.85,745.13,1.786312e+09,2.168589e+08,42184.21,2608.62,0.24,0.01
2,XGBRegressor,72.43,1.89,27391.77,742.68,1.816765e+09,2.430270e+08,42527.46,2860.08,0.23,0.01
3,GradientBoostingRegressor,71.42,1.63,28716.21,735.94,1.880121e+09,2.170082e+08,43287.85,2506.65,0.25,0.01
4,DecisionTreeRegressor,51.07,3.05,36976.56,1114.07,3.217474e+09,3.722200e+08,56627.96,3278.43,0.31,0.00
5,AdaBoostRegressor,2.33,10.20,67371.89,4583.57,6.377578e+09,5.431090e+08,79788.56,3370.97,0.77,0.06


Ao analisar os resultados das métricas de avaliação para os diferentes modelos, observamos que os desempenhos são muito próximos. O XGBoost (XGBRegressor) apresentou um R-Squared médio de 72,43, um MAE médio de 27.391,77 e um RMSE médio de 42.527,46. Esses valores são similares aos obtidos pelos outros modelos testados, como o LGBMRegressor e o RandomForestRegressor.

No entanto, a escolha do XGBoost como o modelo final pode ser justificada por algumas vantagens que ele oferece em relação aos outros modelos. O XGBoost é conhecido por sua eficiência e capacidade de lidar com conjuntos de dados complexos e de grande escala. Ele utiliza algoritmos de boosting e permite a criação de árvores de decisão sequenciais, que são combinadas para formar um modelo mais poderoso.

Além disso, o XGBoost tem a capacidade de lidar com outliers de forma robusta. Ele também oferece recursos de regularização para evitar o overfitting e tem uma implementação eficiente que permite treinamento rápido e predições precisas.

Dessa forma, ao escolher o XGBoost como o modelo final, estamos optando por um modelo versátil, que tem um bom desempenho e pode lidar bem com o nosso problema de regressão. 

# 3.3 - Otimização de Hiperparâmetros

**OBS:** Foram realizados diversos testes utilizando GridSearchCV com diferentes configurações de validação cruzada, como cv=10, cv=5, e também RandomizedSearchCV com cv=10, cv=5, cv=3, num_iter=100, num_iter=50. Todos esses testes apresentaram uma alta demanda computacional.

Ao utilizar a abordagem de estratificação dos dados em 20%, obteve-se sucesso na otimização dos hiperparâmetros. No entanto, é importante ressaltar que essa abordagem pode resultar em uma diminuição na qualidade dos hiperparâmetros encontrados. 

In [14]:
# Dividir os dados em conjunto de treinamento e teste
X_train_subset, _, Y_train_subset, _ = train_test_split(X_train, Y_train, test_size=0.8, random_state=18)

X_train_subset.shape, Y_train_subset.shape

((4115, 30), (4115,))

In [15]:
# Criar o pipeline e definir o parametro grid
PLxgb = Pipeline(steps=[('preprocessor', preprocessor2), ('model', XGBRegressor(random_state=18))])

param_grid = {
    'model__max_depth': [9, 10, 13, 15, 20, 25],
    'model__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'model__n_estimators': [100, 200, 300],
    'model__subsample': [0.8, 1.0],
    'model__colsample_bytree': [0.8, 1.0],
    'model__reg_alpha': [0.0, 0.1, 0.5],
    'model__reg_lambda': [0.0, 0.1, 0.5]
}

# Criar o RandomizedSearchCV object
random_xgb = RandomizedSearchCV(PLxgb, param_distributions=param_grid, cv=5, scoring='r2', n_iter=50, random_state=18, n_jobs=-1, verbose=1)

# Fit the model using the subset of data
random_xgb.fit(X_train_subset, Y_train_subset)

# Print the best score and best parameters
print('\n')
print('Best Score:', random_xgb.best_score_)
print('Best Parameters:', random_xgb.best_params_)
print('\n')


Fitting 5 folds for each of 50 candidates, totalling 250 fits


Best Score: 0.644413683890661
Best Parameters: {'model__subsample': 0.8, 'model__reg_lambda': 0.1, 'model__reg_alpha': 0.0, 'model__n_estimators': 200, 'model__max_depth': 9, 'model__learning_rate': 0.1, 'model__colsample_bytree': 0.8}




In [16]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

# Definir o pipeline com os melhores hiperparâmetros
modelo_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor2),
    ('model', XGBRegressor(
        random_state=18,
        subsample=1.0,
        reg_lambda=0.1,
        reg_alpha=0.0,
        n_estimators=100,
        max_depth=10,
        learning_rate=0.1,
        colsample_bytree=0.8
    ))
])

# Treinar o modelo final
modelo_xgb.fit(X_train, Y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Target Encoder',
                                                  TargetEncoder(),
                                                  ['marca', 'modelo', 'cambio',
                                                   'tipo', 'cor',
                                                   'cidade_vendedor',
                                                   'estado_vendedor',
                                                   'anunciante',
                                                   'tipo_combustivel',
                                                   'blindado', 'tipo_vendedor',
                                                   'entrega_delivery', 'troca',
                                                   'dono_aceita_troca',
                                                   'veiculo_único_dono',
                                             

In [17]:
# Salvar o modelo em disco
joblib.dump(modelo_xgb, 'modelo_xgb.joblib')

['modelo_xgb.joblib']

In [18]:
# Modelo Final

# Definir o pipeline com os melhores hiperparâmetros
modelo_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor2),
    ('model', XGBRegressor(
        random_state=18,
        subsample=1.0,
        reg_lambda=0.1,
        reg_alpha=0.0,
        n_estimators=100,
        max_depth=10,
        learning_rate=0.1,
        colsample_bytree=0.8
    ))
])

# Treinar o modelo final
modelo_xgb.fit(X_train, Y_train)

Y_pred2 = modelo_xgb.predict(X_valid)

# Métricas

RSquared = metrics.r2_score(Y_valid, Y_pred2)*100
MAE = metrics.mean_absolute_error(Y_valid, Y_pred2)
MSE = metrics.mean_squared_error(Y_valid, Y_pred2)
RMSE = np.sqrt(metrics.mean_squared_error(Y_valid, Y_pred2))
MAPE = metrics.mean_absolute_percentage_error(Y_valid, Y_pred2)*100
    
# Resultado Final
    
model_result2 = {'Modelo': ['XGBRegressor'],
                 'R-Squared': [RSquared],
                 'MAE': [MAE],
                 'MSE': [MSE],
                 'RMSE': [RMSE],
                 'MAPE': [MAPE],}

result_final = pd.DataFrame(model_result2, index=['0'])

result_final

,Modelo,R-Squared,MAE,MSE,RMSE,MAPE
0,XGBRegressor,71.09126,27151.4835,2.019115e+09,44934.560163,22.557574


Ao escolher as medidas r2, MAE, MSE, RMSE e MAPE para avaliar o desempenho do modelo de previsão de preço, considerei as seguintes justificativas:

R2 (coeficiente de determinação) indica a proporção da variância explicada pelo modelo.

MAE (Erro Absoluto Médio) é a média das diferenças absolutas entre as previsões e os valores reais.

MSE (Erro Quadrático Médio) é a média dos quadrados das diferenças entre as previsões e os valores reais.

RMSE (Raiz do Erro Quadrático Médio) é a raiz quadrada do MSE e fornece uma medida da dispersão média dos erros.
MAPE (Erro Percentual Absoluto Médio) é a média dos erros percentuais absolutos.

Essas medidas foram selecionadas porque fornecem diferentes perspectivas sobre a qualidade das previsões de preço. O R2 avalia a explicação da variação, o MAE mede a precisão média, o MSE e o RMSE consideram a magnitude e a direção dos erros, e o MAPE avalia a precisão relativa em termos percentuais. Ao utilizar essas medidas em conjunto, é possível obter uma visão abrangente do desempenho do modelo em relação à previsão de preços dos veículos.